# Description

It selects one gene pair (see `Settings` below) and computes the correlation coefficients and p-values across all the tissues in GTEx.
We do this to check whether one pattern found in whole blood also replicates in other tissues.

# Modules

In [1]:
import pandas as pd

from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

from ccc import conf
from ccc.coef import ccc

# Settings

In [2]:
# this gene pair was originally found with ccc on whole blood
# interesting: https://clincancerres.aacrjournals.org/content/26/21/5567.figures-only
gene0_id, gene1_id = "ENSG00000147050.14", "ENSG00000067048.16"
gene0_symbol, gene1_symbol = "KDM6A", "DDX3Y"

CCC_PVALUE_N_PERMS = 1000000

# Paths

In [3]:
TISSUE_DIR = conf.GTEX["DATA_DIR"] / "data_by_tissue"
assert TISSUE_DIR.exists()

In [4]:
OUTPUT_DIR = (
    conf.GTEX["RESULTS_DIR"]
    / "other_tissues"
    / f"{gene0_symbol.lower()}_vs_{gene1_symbol.lower()}"
)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/opt/data/results/gtex_v8/other_tissues/kdm6a_vs_ddx3y')

# Data

## GTEx metadata

In [5]:
gtex_metadata = pd.read_pickle(conf.GTEX["DATA_DIR"] / "gtex_v8-sample_metadata.pkl")

In [6]:
gtex_metadata.shape

(22951, 66)

In [7]:
gtex_metadata.head()

,SUBJID,SEX,AGE,DTHHRDY,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
SAMPID,,,,,,,,,,,,,,,,,,,,,
GTEX-1117F-0003-SM-58Q7G,GTEX-1117F,Female,60-69,4.0,NaN,B1,NaN,NaN,Blood,Whole Blood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTEX-1117F-0003-SM-5DWSB,GTEX-1117F,Female,60-69,4.0,NaN,B1,NaN,NaN,Blood,Whole Blood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTEX-1117F-0003-SM-6WBT7,GTEX-1117F,Female,60-69,4.0,NaN,B1,NaN,NaN,Blood,Whole Blood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTEX-1117F-0011-R10a-SM-AHZ7F,GTEX-1117F,Female,60-69,4.0,NaN,"B1, A1",NaN,NaN,Brain,Brain - Frontal Cortex (BA9),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTEX-1117F-0011-R10b-SM-CYKQ8,GTEX-1117F,Female,60-69,4.0,NaN,"B1, A1",NaN,7.2,Brain,Brain - Frontal Cortex (BA9),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Gene Ensembl ID -> Symbol mapping

In [8]:
gene_map = pd.read_pickle(conf.GTEX["DATA_DIR"] / "gtex_gene_id_symbol_mappings.pkl")

In [9]:
gene_map = gene_map.set_index("gene_ens_id")["gene_symbol"].to_dict()

In [10]:
assert gene_map["ENSG00000145309.5"] == "CABS1"

In [11]:
assert gene_map[gene0_id] == gene0_symbol
assert gene_map[gene1_id] == gene1_symbol

# Compute correlation on all tissues

In [12]:
res_all = pd.DataFrame(
    {
        f.stem.split("_data_")[1]: {
            "cm": ccc(data[gene0_id], data[gene1_id]),
            "pearson": pearsonr(data[gene0_id], data[gene1_id])[0],
            "spearman": spearmanr(data[gene0_id], data[gene1_id])[0],
        }
        for f in TISSUE_DIR.glob("*.pkl")
        if (data := pd.read_pickle(f).T[[gene0_id, gene1_id]].dropna()) is not None
        and data.shape[0] > 10
    }
).T.abs()

In [13]:
res_all.shape

(50, 3)

In [14]:
res_all.head()

,cm,pearson,spearman
colon_transverse,0.280814,0.393881,0.465898
brain_amygdala,0.305677,0.038732,0.154331
artery_coronary,0.242720,0.487610,0.426164
artery_aorta,0.381970,0.579236,0.409761
adrenal_gland,0.188929,0.489400,0.418784


In [15]:
res_all.sort_values("cm")

,cm,pearson,spearman
testis,0.006332,0.017759,0.028708
ovary,0.016698,0.100520,0.125249
vagina,0.016935,0.080397,0.190372
uterus,0.025004,0.078480,0.228621
bladder,0.089510,0.440277,0.216883
small_intestine_terminal_ileum,0.164531,0.361532,0.436642
brain_spinal_cord_cervical_c1,0.166019,0.119269,0.067168
prostate,0.166530,0.576071,0.614194
stomach,0.172034,0.364280,0.453740
adrenal_gland,0.188929,0.489400,0.418784


In [16]:
res_all.sort_values("pearson")

,cm,pearson,spearman
testis,0.006332,0.017759,0.028708
brain_anterior_cingulate_cortex_ba24,0.231620,0.034852,0.188542
brain_amygdala,0.305677,0.038732,0.154331
brain_frontal_cortex_ba9,0.254996,0.054030,0.186901
uterus,0.025004,0.078480,0.228621
vagina,0.016935,0.080397,0.190372
heart_atrial_appendage,0.233004,0.080692,0.049178
ovary,0.016698,0.100520,0.125249
brain_cerebellum,0.219113,0.106469,0.034902
brain_hypothalamus,0.232632,0.118391,0.251149


In [17]:
res_all.sort_values("spearman")

,cm,pearson,spearman
testis,0.006332,0.017759,0.028708
heart_left_ventricle,0.226687,0.191436,0.031302
brain_cerebellum,0.219113,0.106469,0.034902
whole_blood,0.193557,0.237462,0.035350
heart_atrial_appendage,0.233004,0.080692,0.049178
brain_spinal_cord_cervical_c1,0.166019,0.119269,0.067168
brain_cortex,0.198577,0.156560,0.107833
ovary,0.016698,0.100520,0.125249
adipose_visceral_omentum,0.194845,0.289887,0.137254
brain_amygdala,0.305677,0.038732,0.154331


# Compute p-values on all tissues

In [18]:
res_pval_all = pd.DataFrame(
    {
        f.stem.split("_data_")[1]: {
            "cm": ccc(
                data[gene0_id],
                data[gene1_id],
                pvalue_n_perms=CCC_PVALUE_N_PERMS,
                n_jobs=conf.GENERAL["N_JOBS"],
            )[1],
            "pearson": pearsonr(data[gene0_id], data[gene1_id])[1],
            "spearman": spearmanr(data[gene0_id], data[gene1_id])[1],
        }
        for f in TISSUE_DIR.glob("*.pkl")
        if (data := pd.read_pickle(f).T[[gene0_id, gene1_id]].dropna()) is not None
        and data.shape[0] > 10
    }
).T.abs()

In [19]:
res_pval_all.shape

(50, 3)

In [20]:
res_pval_all.head()

,cm,pearson,spearman
colon_transverse,9.999990e-07,1.613504e-16,2.880714e-23
brain_amygdala,9.999990e-07,6.356755e-01,5.764275e-02
artery_coronary,9.999990e-07,9.731623e-16,5.220895e-12
artery_aorta,9.999990e-07,4.513966e-40,6.380372e-19
adrenal_gland,9.999990e-07,6.058615e-17,2.230048e-12


In [21]:
res_pval_all.sort_values("cm")

,cm,pearson,spearman
colon_transverse,9.999990e-07,1.613504e-16,2.880714e-23
prostate,9.999990e-07,4.637571e-23,8.471438e-27
brain_frontal_cortex_ba9,9.999990e-07,4.371643e-01,6.734313e-03
brain_caudate_basal_ganglia,9.999990e-07,7.044831e-03,6.797072e-04
muscle_skeletal,9.999990e-07,3.656202e-13,1.563020e-07
stomach,9.999990e-07,1.045907e-12,1.240450e-19
skin_not_sun_exposed_suprapubic,9.999990e-07,4.784340e-21,3.167647e-11
adipose_visceral_omentum,9.999990e-07,6.191569e-12,1.373269e-03
nerve_tibial,9.999990e-07,6.073870e-75,1.130228e-25
pancreas,9.999990e-07,1.432674e-08,4.739023e-06


In [22]:
res_pval_all.sort_values("pearson")

,cm,pearson,spearman
nerve_tibial,9.999990e-07,6.073870e-75,1.130228e-25
cells_cultured_fibroblasts,9.999990e-07,1.188516e-73,3.662235e-49
artery_tibial,9.999990e-07,5.248493e-71,3.246061e-25
thyroid,9.999990e-07,4.844271e-53,1.541651e-30
artery_aorta,9.999990e-07,4.513966e-40,6.380372e-19
adipose_subcutaneous,9.999990e-07,6.287645e-40,2.618709e-17
esophagus_muscularis,9.999990e-07,8.650204e-33,8.375537e-18
colon_sigmoid,9.999990e-07,2.450898e-28,6.395478e-17
skin_sun_exposed_lower_leg,9.999990e-07,5.836576e-27,2.245602e-16
esophagus_mucosa,9.999990e-07,6.374962e-27,2.030847e-21


In [23]:
res_pval_all.sort_values("spearman")

,cm,pearson,spearman
cells_cultured_fibroblasts,9.999990e-07,1.188516e-73,3.662235e-49
thyroid,9.999990e-07,4.844271e-53,1.541651e-30
prostate,9.999990e-07,4.637571e-23,8.471438e-27
nerve_tibial,9.999990e-07,6.073870e-75,1.130228e-25
artery_tibial,9.999990e-07,5.248493e-71,3.246061e-25
colon_transverse,9.999990e-07,1.613504e-16,2.880714e-23
esophagus_mucosa,9.999990e-07,6.374962e-27,2.030847e-21
stomach,9.999990e-07,1.045907e-12,1.240450e-19
artery_aorta,9.999990e-07,4.513966e-40,6.380372e-19
esophagus_muscularis,9.999990e-07,8.650204e-33,8.375537e-18


# Save

## Coefficient values

In [24]:
res_all.to_pickle(OUTPUT_DIR / "coef_values.pkl")

## Coefficient p-values

In [25]:
res_pval_all.to_pickle(OUTPUT_DIR / "coef_pvalues.pkl")